 
Denoising using RNN- LSTM

Simple structure with 1 LSTM And followed by one dense layer taken both with 513 hidden units.


In [0]:
import matplotlib.pyplot as plt
import numpy as np
import math
import librosa
import pickle
import tensorflow as tf
import os

As the fileset was big, and colaboratory suffers from temporary storage, files were stored in drive and then mounted from there. 

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [0]:
def getSNR(RawInp,Pred):
    t = min(len(RawInp),len(Pred))
    clippedY_True = RawInp[:t]
    clippedY_Pred = Pred[:t]
    signal  = np.dot(clippedY_True,clippedY_True.T)
    subtract = clippedY_True - clippedY_Pred
    noise = np.dot(subtract,subtract.T)
    
    return 10*math.log10(signal/noise)

the following are the path combinations for the folders

In [0]:
path = './gdrive/My Drive/homework3/'

path_train = path + 'tr/'
path_val = path + 'v/'
path_test = path + 'te/'

path_result = path + 'result/'
path_model = path + 'model/'

In [0]:
def load_wav(file,f=0):
  sn, sr=librosa.load(file, sr=None)
  Xdata1=librosa.stft(sn, n_fft=1024, hop_length=512)
  if f==1: return sn,sr,Xdata1.T
  return Xdata1.T
  
def numtostring(i):
  if i!=0 : return "0"*(4 - (math.floor(math.log10(i)) + 1))+str(i)
  else: return "0000"

nums = [numtostring(i) for i in range(1200)]


In [0]:
Xtr, Str, Ntr = [], [], []
Xv, Sv, Nv = [], [], []
max_width = 513
max_length = 200

the following can be un-commented when pickle file not there.
As colab is slow pickle was used to load everytime.

In [0]:
"""
for i in nums:
  Xtr.append(load_wav(path_train + "trx"+i+".wav",0))
  Str.append(load_wav(path_train+ "trs"+i+".wav",0))
  Ntr.append(load_wav(path_train+"trn"+i+".wav",0))
train=[Xtr,Str,Ntr]
with open(path+'train.pickle', 'wb') as f:
  pickle.dump(train, f)
"""

'\nfor i in nums:\n  Xtr.append(load_wav(path_train + "trx"+i+".wav",0))\n  Str.append(load_wav(path_train+ "trs"+i+".wav",0))\n  Ntr.append(load_wav(path_train+"trn"+i+".wav",0))\ntrain=[Xtr,Str,Ntr]\nwith open(path+\'train.pickle\', \'wb\') as f:\n  pickle.dump(train, f)\n'

In [0]:
"""
snv= []

for i in nums:  
  Xv.append(load_wav(path_val+"vx"+i+".wav",0))
  a,b,c = load_wav(path_val+"vs"+i+".wav",1)
  snv.append(a),Sv.append(c)  
  Nv.append(load_wav(path_val+"vn"+i+".wav",0))
val = [snv,Xv,Sv,Nv]
with open(path+'val.pickle', 'wb') as f:
  pickle.dump(val, f)
"""


'\nsnv= []\n\nfor i in nums:  \n  Xv.append(load_wav(path_val+"vx"+i+".wav",0))\n  a,b,c = load_wav(path_val+"vs"+i+".wav",1)\n  snv.append(a),Sv.append(c)  \n  Nv.append(load_wav(path_val+"vn"+i+".wav",0))\nval = [snv,Xv,Sv,Nv]\nwith open(path+\'val.pickle\', \'wb\') as f:\n  pickle.dump(val, f)\n'

In [0]:
"""
sns,srs,test_X = [], [], []
for i in range(400):
  a,b,c =load_wav(path_test+"tex"+numtostring(i)+".wav",1)
  sns.append(a),test_X.append(c)
test= [sns,test_X]
with open(path+'test.pickle', 'wb') as f:
    pickle.dump(test, f)
"""

'\nsns,srs,test_X = [], [], []\nfor i in range(400):\n  a,b,c =load_wav(path_test+"tex"+numtostring(i)+".wav",1)\n  sns.append(a),test_X.append(c)\ntest= [sns,test_X]\nwith open(path+\'test.pickle\', \'wb\') as f:\n    pickle.dump(test, f)\n'

In [0]:
import pickle
with open(path+'train.pickle', 'rb') as f:
  loaded_train = pickle.load(f)
with open(path+'test.pickle', 'rb') as f:
  loaded_test = pickle.load(f)  
with open(path+'val.pickle', 'rb') as f:
  loaded_val = pickle.load(f)

In [0]:
def get_abs(stft_list):
  out = []
  for x in stft_list:
    out.append(np.abs(np.pad(x, [(0, max_length-x.shape[0]), (0, 0)],mode='constant')))
  return np.array(out)             


In [0]:
loaded_X,loaded_S,loaded_N = loaded_train
mloaded_X,mloaded_S,mloaded_N = [get_abs(x) for x in loaded_train]
target = 1 * (mloaded_S > mloaded_N)

sn,test_X = loaded_test
snvv, val_X,val_S,val_N = loaded_val
mval_X,mval_S,mval_N = [get_abs(x) for x in loaded_val[1:]]

In [0]:
train_tot = len(loaded_X)
batch_size = 128
num_features = 513
num_hidden = 513

learning_rate = 0.007
num_epochs = 500
X = tf.placeholder(tf.float32, [None, max_length, num_features])
Y = tf.placeholder(tf.float32, [None, max_length, num_features])
keep_prob = tf.placeholder(tf.float32, name='keep_prob')


In [0]:

lstm = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_hidden),output_keep_prob=keep_prob)
output, state = tf.nn.dynamic_rnn(lstm, X, dtype=tf.float32)
dense = tf.layers.dense(inputs=output,units=513, activation=tf.nn.sigmoid)
sess = tf.Session()
saver = tf.train.Saver()
loss = tf.losses.mean_squared_error(labels=Y, predictions=dense)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss=loss)
init = tf.global_variables_initializer()


Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.


The following is the training phase with number of epochs set to be 100 initially.

In [0]:
#saver.restore(sess, path_model + 'model_lstm.ckpt')

In [0]:
sess.run(tf.global_variables_initializer())

#
for epoch in range(num_epochs):
  loss_tot = 0
  for i in range(0, train_tot, batch_size):
    end = min(i + batch_size, train_tot)    
    batch_x = mloaded_X[i:end]
    batch_y = target[i:end]
    _, lossi = sess.run([train, loss], feed_dict={X: batch_x, Y: batch_y,keep_prob: 0.9})
    loss_tot += lossi
    
  print(epoch, loss_tot)
saver.save(sess, path_model + 'model_lstm.ckpt')

0 1.9160180389881134
1 1.1862040236592293
2 1.0933885648846626
3 1.0324436724185944
4 0.9769348204135895
5 0.9359132423996925
6 0.9098641574382782
7 0.8893876522779465
8 0.8650663867592812
9 0.8227577731013298
10 0.7963450998067856
11 0.7805962711572647
12 0.7981526330113411
13 0.7917355671525002
14 0.7617887929081917
15 0.7462948560714722
16 0.7331531494855881
17 0.7275565788149834
18 0.7243918403983116
19 0.7251408845186234
20 0.7178953513503075
21 0.7055901400744915
22 0.6924942433834076
23 0.6792283691465855
24 0.669565174728632
25 0.6610085181891918
26 0.6541378833353519
27 0.651338592171669
28 0.6447532027959824
29 0.6484431438148022
30 0.6514732986688614
31 0.6707202158868313
32 0.6561906151473522
33 0.645463727414608
34 0.6389500088989735
35 0.6411176696419716
36 0.646407350897789
37 0.6552474498748779
38 0.6595397591590881
39 0.648781094700098
40 0.6333590112626553
41 0.6213591396808624
42 0.6090425550937653
43 0.6015408001840115
44 0.5967862233519554
45 0.5941989421844482
46 

'./gdrive/My Drive/homework3/model/model_lstm.ckpt'

Following is the validation step, where the trained model is used to calculate for the validation files and the SNR is calculated to observe the performance.

In [0]:
val_tot = len(mval_X)
snr= [] 
for i in range(0, val_tot, batch_size):
  end = i + batch_size if val_tot>i + batch_size else val_tot
  batch_x = mval_X[i:end]

  m_pred = sess.run([dense], feed_dict={X: batch_x,keep_prob:1})
  #print(m_pred[0].shape)
  for j in range(i, end):
    x = val_X[j]
    s_val = mval_S[j][:val_X[j].shape[0], :]
    m = m_pred[0][j - i][:val_X[j].shape[0], :]
    m = 1*(m>0.5)
    cleaned = x * m
    snr.append(getSNR(snvv[j],librosa.istft(cleaned.T, hop_length=512)))
        
print(np.mean(snr))

11.078615647458843


Finally, the test set of files are passed to the trained network, to get the desired IBM output, which is used to get the correct signal back.

In [0]:
mtest_X = get_abs(test_X)
test_tot = len(test_X)

for i in range(0, test_tot, batch_size):
  end = i + batch_size if test_tot>i + batch_size else test_tot
  
  batch_x = mtest_X[i:end]
   
  m_pred = sess.run([dense], feed_dict={X: batch_x,keep_prob:1})
  M_hat = m_pred[0]
  for j in range(i, end):
    x = test_X[j]
    s = mtest_X[j][:test_X[j].shape[0], :]
    m = M_hat[j - i][:test_X[j].shape[0], :]
    m = 1*(m>0.5)
    cleaned = x * m
    if not os.path.exists(path_result):
      os.makedirs(path_result)
    file = path_result + 'cleaned_test' + numtostring(j) + '.wav'
    s_hat = librosa.istft(cleaned.T, hop_length=512)
    librosa.output.write_wav(file, s_hat, sr=16000)